## Install Dependencies and Initial Plan


1. Model Setup (Ollama + DeepSeek)

2. Data Preparation (Knowledge Source)

3. Chunk & Embed Data

4. Build Vector DB

5. Query Pipeline (RAG Flow)

6. Streamlit Setup (UI)

In [10]:
!pip install langchain langchain-community langchain-core



## Model Setup (Ollama + DeepSeek)

In [ ]:
# Before Running the cell have a look on readme file and follow instruction

from langchain_ollama import OllamaLLM

# Need to run ollama in terminal before running the cell
llm = OllamaLLM(model="deepseek-r1:1.5b")

# Ask a question
response = llm.invoke("What is Rag?")
print("🤖", response)


🤖 <think>

</think>

Rag is a term often used metaphorically or technically to describe something that is small, insignificant, or of low value. It can also refer to clothing made from ragged (dimpled) material, such as rag sheets, especially in the context of producing large amounts of fabric. In a more abstract sense, it might be used to denote something that lacks substance, purpose, or quality.

If you're referring to something specific, feel free to provide more context!


## Data Preparation (Knowledge Source)

In [ ]:
#Create .txt, .pdf, or .md file (example: docs/my_faqs.txt)

#Keep clean, structured knowledge (questions, answers, docs)






## Chunk & Embed Data

In [ ]:
#Use LangChain to:

#Load your documents

#Split into smaller chunks

#Embed with HuggingFaceEmbeddings

#Store into Chroma vector DB

## Build Vector DB



In [ ]:
Chroma.from_documents(
    documents=split_docs,
    embedding=HuggingFaceEmbeddings(...),
    persist_directory="./chroma_db"
)


# working on this to find solution

## Query Pipeline (RAG Flow)



In [ ]:
#Accept user question

#Convert to embedding

#Search top-k similar chunks from Chroma

#Concatenate chunks + question

#Send to OllamaLLM via LangChain

#Get DeepSeek response

## Streamlit Setup (UI)